### 구글 드라이브 마운트

In [1]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# 구글 드라이브 경로 설정
%cd /content/drive/MyDrive/텍스트 프로젝트
%pwd

Mounted at /content/drive
/content/drive/MyDrive/텍스트 프로젝트


'/content/drive/MyDrive/텍스트 프로젝트'

### 라이브러리 설치

In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 125.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.1 MB/s eta 0:00:00


### 라이브러리 불러오기

In [42]:
import torch
from transformers import AdamW, AutoConfig, GPT2LMHeadModel, PreTrainedTokenizerFast
from transformers import BertTokenizer, BertForSequenceClassification, pipeline


### 가사 생성

In [92]:
device = torch.device("cpu")

tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                pad_token='<pad>', mask_token='<mask>')


# 저장한 모델 불러오기
load_data = torch.load("model_kogpt2/kogpt2_fine-tunning_51.pt", map_location=torch.device("cpu"))

# 모델 로드
config = AutoConfig.from_pretrained(
            "gpt2",
            vocab_size=len(tokenizer),
            bos_token_id = tokenizer.bos_token_id,
            eos_token_id = tokenizer.eos_token_id,
        )

model = GPT2LMHeadModel(config).to(device)
model.load_state_dict(load_data['model'])

# 옵티마이저 로드
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8)
optimizer.load_state_dict(load_data['optimizer'])

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [93]:
def generate(keyword):
    input_ids = tokenizer.encode(keyword, return_tensors='pt')
    input_ids = input_ids.to(device)
    gen_ids = model.generate(input_ids,
                        max_length=128,
                        repetition_penalty=2.0,
                        pad_token_id=tokenizer.pad_token_id,
                        eos_token_id=tokenizer.eos_token_id,
                        bos_token_id=tokenizer.bos_token_id,
                        use_cache=True)
    generated = tokenizer.decode(gen_ids[0])
    return generated

In [94]:
keywords = ['사랑', '그대', '생각', '사람']

lyrics_lst = []

for keyword in keywords:
    #keyword = keyword.to(device)
    gen = generate(keyword)
    gen_print = gen.split('\n')[0:2][0]
    lyrics_lst.append(gen_print)
    print(f"{keyword} : {gen_print}")

사랑 : 사랑이별가요 괜찮아 질거야 이별없는 거니까그 많은 약속들은 어제와 같은 선물로점쳐지고 떠나가는 게제 뻔한 일이죠하지만 내 진심만은 알아줬으면 해요한마디 말이면 충분해나 혹시 그대 생각 나겠지만사랑에 바쁜 나를 피해서저별을 준비했단걸사랑이 중요하진 않아잘 안돼 우리가 왜 이렇게 외로운지난 몰라 당신만을 위해서만 기도하고바랬던 사랑이 필요했다면다른 건 아무 의미없어나는 그대를 알기 때문에살아가면서 한번도 느껴지지 않던자존심을 다 버렸
그대 : 그대 사랑했단 거오래 전에 얘기지노을처럼 피어난가슴 태우던 사랑을그대 떠나 가던 밤모두 잊으라시며마지막 인사를 나 하염없이 하던 그 약속간절한 내 마음약속 하지 않았네기다리는 나를 모르시나요행여나 찾아 올까봐 혼자 애태우는 이 마음은어느샌가 거리길가에 스쳐가는언제나 익숙한 목소리작은 두 글자로우리 이름을 써놓고몇 밤을 울다 잠든 다정히 창밖 바라보기만 보며 말하던 나이별을 두고이제는 떠난 사람무엇인가 빈 추억인거죠사랑했던 우리야 이별
생각 : 생각도 못했던 말내겐 사랑이란 걸 몰랐던거야이제 우리 같은 시간 속을 남처럼그렇게 걸어왔어요생각보다 오래 된 것 같아요한 번쯤은 나를 떠나줄 것만 같아서난 정말 쉬지 않았죠사랑했던 날 늘 붙잡고떠나지 말라고흘러댔었던 내 맘을하얗게 물들인 밤하늘에 그려보아직도 나는 서 있을까더는 네 모습 볼 수 없겠지만아직 너를 위해 준비해둔 말이제는 말해볼 만하지 못한 나의 마음나는 너무 아파다시 널 보낼 자신이 없어초라했나봐 넌 내게 언제나 이런 얘기를 했었지어
사람 : 사람 사랑했단 거오래 전에 얘기지노을처럼 피어난가슴 태우던그 이름 난사랑하네 다정한 그 목소리로노래 불러요잊어 주세요가끔은 그리운 사람과해맑고 웃으며내게 편지를 쓰나봐오늘 밤도 널 그리워하다못 잊어서 돌아오질 못해이렇게 나는 울었네이별 후 더 힘들다 맘 이젠 추억아직 허기진 조금 남아있우리 다시 만나면감사예전 그대 모습 그대로 일거야안녕다시 또 우리 만날 때까지몇 번이고 말해줘나의 목소리 너의 미소다가올 이별마저 안 될 만큼

In [96]:
lyrics_lst

['사랑이별가요 괜찮아 질거야 이별없는 거니까그 많은 약속들은 어제와 같은 선물로점쳐지고 떠나가는 게제 뻔한 일이죠하지만 내 진심만은 알아줬으면 해요한마디 말이면 충분해나 혹시 그대 생각 나겠지만사랑에 바쁜 나를 피해서저별을 준비했단걸사랑이 중요하진 않아잘 안돼 우리가 왜 이렇게 외로운지난 몰라 당신만을 위해서만 기도하고바랬던 사랑이 필요했다면다른 건 아무 의미없어나는 그대를 알기 때문에살아가면서 한번도 느껴지지 않던자존심을 다 버렸',
 '그대 사랑했단 거오래 전에 얘기지노을처럼 피어난가슴 태우던 사랑을그대 떠나 가던 밤모두 잊으라시며마지막 인사를 나 하염없이 하던 그 약속간절한 내 마음약속 하지 않았네기다리는 나를 모르시나요행여나 찾아 올까봐 혼자 애태우는 이 마음은어느샌가 거리길가에 스쳐가는언제나 익숙한 목소리작은 두 글자로우리 이름을 써놓고몇 밤을 울다 잠든 다정히 창밖 바라보기만 보며 말하던 나이별을 두고이제는 떠난 사람무엇인가 빈 추억인거죠사랑했던 우리야 이별',
 '생각도 못했던 말내겐 사랑이란 걸 몰랐던거야이제 우리 같은 시간 속을 남처럼그렇게 걸어왔어요생각보다 오래 된 것 같아요한 번쯤은 나를 떠나줄 것만 같아서난 정말 쉬지 않았죠사랑했던 날 늘 붙잡고떠나지 말라고흘러댔었던 내 맘을하얗게 물들인 밤하늘에 그려보아직도 나는 서 있을까더는 네 모습 볼 수 없겠지만아직 너를 위해 준비해둔 말이제는 말해볼 만하지 못한 나의 마음나는 너무 아파다시 널 보낼 자신이 없어초라했나봐 넌 내게 언제나 이런 얘기를 했었지어',
 '사람 사랑했단 거오래 전에 얘기지노을처럼 피어난가슴 태우던그 이름 난사랑하네 다정한 그 목소리로노래 불러요잊어 주세요가끔은 그리운 사람과해맑고 웃으며내게 편지를 쓰나봐오늘 밤도 널 그리워하다못 잊어서 돌아오질 못해이렇게 나는 울었네이별 후 더 힘들다 맘 이젠 추억아직 허기진 조금 남아있우리 다시 만나면감사예전 그대 모습 그대로 일거야안녕다시 또 우리 만날 때까지몇 번이고 말해줘나의 목소리 너의 미소다가올 이별마저 안 될 만큼너를 많이 

### 장르 예측

In [97]:
# 한국어 BERT 중 하나인 'klue/bert-base'를 사용.
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

In [98]:
num_labels = 14

model = BertForSequenceClassification.from_pretrained("klue/bert-base",
        num_labels=num_labels, problem_type="multi_label_classification")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [99]:
# 모델 로드
epoch = 14
model.load_state_dict(torch.load(f"model_bert/BERT_multilabel_model_{epoch}.pt", map_location=torch.device("cpu")))

<All keys matched successfully>

In [100]:
#pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0, max_length=512,
#                return_all_scores=True, function_to_apply='sigmoid')

pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, max_length=512,
                return_all_scores=True, function_to_apply='sigmoid')

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [101]:
f = open('model_bert/genre_to_idx.txt', 'r')     # mode = 부분은 생략해도 됨
genre_to_idx = f.read()
genre_to_idx = eval(genre_to_idx)

In [102]:
for idx, val in enumerate(genre_to_idx):
    print(idx, val)

0 일렉트로니카
1 성인가요/트로트
2 애니메이션/웹툰
3 포크/블루스
4 록/메탈
5 국내드라마
6 인디음악
7 만화
8 키즈
9 랩/힙합
10 R&B/Soul
11 댄스
12 발라드
13 국내영화


In [103]:
label_dict = {f"LABEL_{idx}" : val for idx, val  in enumerate(genre_to_idx)}

In [104]:
def prediction(text):
    result = pipe(text)
    return [label_dict[res['label']] for res in result[0] if res['score'] > 0.5]

In [105]:
import re
# 가사 전처리
def preprocess(sentences):
    lst = []
    for sentence in sentences:
        # \n과 \t를 제거한다
        sentence = re.sub('\\t', '', sentence)
        sentence = re.sub('\\n', '', sentence)
        # 한국어, 공백 빼고 제거하기
        sentence = re.sub('[^ㄱ-ㅎ가-힣\s]+', '', sentence)
        # 문장 양옆의 띄어쓰기를 지운다
        sentence = sentence.strip()

        lst.append(sentence)
    return lst

In [106]:
# lyrics_lst : 모델로 생성된 가사

lyrics_genre = []

for num, lyrics in enumerate(lyrics_lst):
  genre = prediction([lyrics])
  lyrics_genre.append([lyrics, genre])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [107]:
lyrics_genre

[['사랑이별가요 괜찮아 질거야 이별없는 거니까그 많은 약속들은 어제와 같은 선물로점쳐지고 떠나가는 게제 뻔한 일이죠하지만 내 진심만은 알아줬으면 해요한마디 말이면 충분해나 혹시 그대 생각 나겠지만사랑에 바쁜 나를 피해서저별을 준비했단걸사랑이 중요하진 않아잘 안돼 우리가 왜 이렇게 외로운지난 몰라 당신만을 위해서만 기도하고바랬던 사랑이 필요했다면다른 건 아무 의미없어나는 그대를 알기 때문에살아가면서 한번도 느껴지지 않던자존심을 다 버렸',
  ['R&B/Soul']],
 ['그대 사랑했단 거오래 전에 얘기지노을처럼 피어난가슴 태우던 사랑을그대 떠나 가던 밤모두 잊으라시며마지막 인사를 나 하염없이 하던 그 약속간절한 내 마음약속 하지 않았네기다리는 나를 모르시나요행여나 찾아 올까봐 혼자 애태우는 이 마음은어느샌가 거리길가에 스쳐가는언제나 익숙한 목소리작은 두 글자로우리 이름을 써놓고몇 밤을 울다 잠든 다정히 창밖 바라보기만 보며 말하던 나이별을 두고이제는 떠난 사람무엇인가 빈 추억인거죠사랑했던 우리야 이별',
  ['성인가요/트로트']],
 ['생각도 못했던 말내겐 사랑이란 걸 몰랐던거야이제 우리 같은 시간 속을 남처럼그렇게 걸어왔어요생각보다 오래 된 것 같아요한 번쯤은 나를 떠나줄 것만 같아서난 정말 쉬지 않았죠사랑했던 날 늘 붙잡고떠나지 말라고흘러댔었던 내 맘을하얗게 물들인 밤하늘에 그려보아직도 나는 서 있을까더는 네 모습 볼 수 없겠지만아직 너를 위해 준비해둔 말이제는 말해볼 만하지 못한 나의 마음나는 너무 아파다시 널 보낼 자신이 없어초라했나봐 넌 내게 언제나 이런 얘기를 했었지어',
  ['발라드']],
 ['사람 사랑했단 거오래 전에 얘기지노을처럼 피어난가슴 태우던그 이름 난사랑하네 다정한 그 목소리로노래 불러요잊어 주세요가끔은 그리운 사람과해맑고 웃으며내게 편지를 쓰나봐오늘 밤도 널 그리워하다못 잊어서 돌아오질 못해이렇게 나는 울었네이별 후 더 힘들다 맘 이젠 추억아직 허기진 조금 남아있우리 다시 만나면감사예전 그대 모습 그대로 일거야안녕다시 또 우

### 예측한 장르로 곡제목 생성

In [108]:
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2').to(device)

optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8)

load_data = torch.load("model_kogpt2/genre_song/kogpt2_fine-tunning_13.pt", map_location=torch.device("cpu"))


# 저장한 모델 불러오기
# 모델 로드
model.load_state_dict(load_data['model'])

# 옵티마이저 로드
optimizer.load_state_dict(load_data['optimizer'])

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [109]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
                bos_token='</s>', eos_token='</s>', unk_token='<unk>',
                pad_token='<pad>', mask_token='<mask>')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [110]:
def generate(keyword):
    input_ids = tokenizer.encode(keyword, return_tensors='pt')
    input_ids = input_ids.to(device)
    gen_ids = model.generate(input_ids,
                        max_length=128,
                        repetition_penalty=2.0,
                        pad_token_id=tokenizer.pad_token_id,
                        eos_token_id=tokenizer.eos_token_id,
                        bos_token_id=tokenizer.bos_token_id,
                        use_cache=True)
    generated = tokenizer.decode(gen_ids[0])
    pad_idx = generated.find('<pad>')
    generated = generated[:pad_idx]
    return generated

In [112]:
lyrics_genre[0][1][0]

'R&B/Soul'

In [113]:
genre_lst = [one_row[1][0] for one_row in lyrics_genre]

In [114]:
gen_lst = []
for keyword in genre_lst:
  gen = generate(keyword)
  print(f"{gen}")
  gen_lst.append(gen)

R&B/Soul: 너를 그리다
성인가요/트로트: 내사랑 그대에게
발라드: 사랑합니다...
발라드: 사랑합니다...


In [115]:
gen_lst

['R&B/Soul: 너를 그리다', '성인가요/트로트: 내사랑 그대에게', '발라드: 사랑합니다...', '발라드: 사랑합니다...']

In [116]:
lyrics_genre

[['사랑이별가요 괜찮아 질거야 이별없는 거니까그 많은 약속들은 어제와 같은 선물로점쳐지고 떠나가는 게제 뻔한 일이죠하지만 내 진심만은 알아줬으면 해요한마디 말이면 충분해나 혹시 그대 생각 나겠지만사랑에 바쁜 나를 피해서저별을 준비했단걸사랑이 중요하진 않아잘 안돼 우리가 왜 이렇게 외로운지난 몰라 당신만을 위해서만 기도하고바랬던 사랑이 필요했다면다른 건 아무 의미없어나는 그대를 알기 때문에살아가면서 한번도 느껴지지 않던자존심을 다 버렸',
  ['R&B/Soul']],
 ['그대 사랑했단 거오래 전에 얘기지노을처럼 피어난가슴 태우던 사랑을그대 떠나 가던 밤모두 잊으라시며마지막 인사를 나 하염없이 하던 그 약속간절한 내 마음약속 하지 않았네기다리는 나를 모르시나요행여나 찾아 올까봐 혼자 애태우는 이 마음은어느샌가 거리길가에 스쳐가는언제나 익숙한 목소리작은 두 글자로우리 이름을 써놓고몇 밤을 울다 잠든 다정히 창밖 바라보기만 보며 말하던 나이별을 두고이제는 떠난 사람무엇인가 빈 추억인거죠사랑했던 우리야 이별',
  ['성인가요/트로트']],
 ['생각도 못했던 말내겐 사랑이란 걸 몰랐던거야이제 우리 같은 시간 속을 남처럼그렇게 걸어왔어요생각보다 오래 된 것 같아요한 번쯤은 나를 떠나줄 것만 같아서난 정말 쉬지 않았죠사랑했던 날 늘 붙잡고떠나지 말라고흘러댔었던 내 맘을하얗게 물들인 밤하늘에 그려보아직도 나는 서 있을까더는 네 모습 볼 수 없겠지만아직 너를 위해 준비해둔 말이제는 말해볼 만하지 못한 나의 마음나는 너무 아파다시 널 보낼 자신이 없어초라했나봐 넌 내게 언제나 이런 얘기를 했었지어',
  ['발라드']],
 ['사람 사랑했단 거오래 전에 얘기지노을처럼 피어난가슴 태우던그 이름 난사랑하네 다정한 그 목소리로노래 불러요잊어 주세요가끔은 그리운 사람과해맑고 웃으며내게 편지를 쓰나봐오늘 밤도 널 그리워하다못 잊어서 돌아오질 못해이렇게 나는 울었네이별 후 더 힘들다 맘 이젠 추억아직 허기진 조금 남아있우리 다시 만나면감사예전 그대 모습 그대로 일거야안녕다시 또 우